<a href="https://colab.research.google.com/github/kushv16/Audio-Sentiment-Analysis/blob/master/Models_Trained_on_Combo_of_JL_and_EMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [8]:
# import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format='retina'
import warnings
warnings.filterwarnings("ignore")

# Import the dataset with the features
df = pd.read_csv('/content/drive/MyDrive/LY Project/combo.csv')

In [7]:
df['dataset'].unique()

array(['RAVDESS', 'CREMA-D', 'SAVEE', 'TESS', 'JL-Corpus', 'EmoV_DB'],
      dtype=object)

In [9]:
include_datasets = ['EmoV_DB', 'JL-Corpus']

# Convert target to numerical values
df['emotion'].replace({'angry':0,'disgust':1,'fear':2,'happy':3,'neutral':4,'sad':5,'surprise':6}, inplace=True)
df['gender'].replace({'female':0,'male':1}, inplace=True)
df.dropna(inplace = True)
df = df[df['dataset'].isin(include_datasets)]
# Create feature matrix (X) and numerical target (y)
X = df.drop(columns=['path','filename','dataset','duration','age','emotion','sample_rate'])
y = df.emotion.values

In [11]:
df['dataset'].unique()

array(['JL-Corpus', 'EmoV_DB'], dtype=object)

In [12]:
from sklearn.model_selection import train_test_split

# Create train and test set
X_tr, X_te, y_tr, y_te = train_test_split(X, y, train_size=0.9, shuffle=True, stratify=y, random_state=0)

# Checking size
print('Shape of X_tr: ', X_tr.shape)
print('Shape of y_tr: ', y_tr.shape)
print('Shape of X_te: ', X_te.shape)
print('Shape of y_te: ', y_te.shape)

Shape of X_tr:  (4333, 90)
Shape of y_tr:  (4333,)
Shape of X_te:  (482, 90)
Shape of y_te:  (482,)


In [13]:
# Scale the data
from sklearn.preprocessing import StandardScaler

# Create Standard scaler
scaler = StandardScaler()

# Rescale data
X_tr_rescaled = scaler.fit_transform(X_tr)
X_te_rescaled = scaler.transform(X_te)

In [15]:
%%time

# importing necessary libraries for this task
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=500, max_depth=100, class_weight='balanced', random_state=0)
rf.fit(X_tr_rescaled,y_tr)

print("Test accuracy for the random forest model after the tuning: {:.3f}".format(rf.score(X_te_rescaled,y_te)))

Test accuracy for the random forest model after the tuning: 0.954
CPU times: user 20.3 s, sys: 4.12 ms, total: 20.3 s
Wall time: 21.8 s


In [16]:
from sklearn.svm import SVC

best_svm = SVC(kernel='rbf', C=100, gamma=0.01, probability=True)

best_svm.fit(X_tr_rescaled,y_tr)

print('Test accuracy of the SVC classifier with RBF kernel: {:.3f}'.format(best_svm.score(X_te_rescaled, y_te)))

Test accuracy of the SVC classifier with RBF kernel: 0.990


In [18]:
from sklearn.tree import DecisionTreeClassifier

best_decision_tree = DecisionTreeClassifier(max_leaf_nodes=99, min_samples_split=3)

best_decision_tree.fit(X_tr_rescaled,y_tr)

print('Test accuracy of the Decision Tree classifier: {:.3f}'.format(best_decision_tree.score(X_te_rescaled, y_te)))

Test accuracy of the Decision Tree classifier: 0.846
